In [46]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
#tx = "f0f20503000100000001008145000003000000130009000000090191640114000103643700010000005500c8001e00320000080000000f10055f0000000303000f016300010100004c617474652020202020202020202020202020003200000005001701010101640114000104643800010000004600be002300320000080014000f10055f0000050303000f0134000102000043617070756363696e6f20202020202020202000320000000500d80009010164011400010364370001000000e60069000000320100080000000f32055f0000000303000f01410001000000576869746520436f666665652020202020202000320000000500340003010164010000010064390001000000180100000000320100080000000f32055f0000000000000f01460001000000416d65726963616e6f2020202020202020202000320000000500c6001b010164011400010964340001000000230064000000320000080014000f10055f3400050300010f014100010200004d6f6368612020202020202020202020202020003200aa0005002a01200501640100000009641e00000001000a0069000000000100080000000f1000012d00000000010a010a0001000000486f742043686f636f6c617465202020202020003200be0000000801120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000130201640100000000642100000000000a00b4000000000000080000000f1000010000000000020a01140001000000686f7420776174657220202020202020202020003200000005003110120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000020101640114000104644c0001000000420096001e00320000080014000f32055f0000050303000f012b0001020000322043617070756363696e6f20202020202020003200000005000000080101640114000100645a0001000000460000000000320000080014000f32055f0000050000000f012300010200003220657370726573736f2020202020202020200032000000050000000a0101650114000103643d00010000003c00ae002d00320000080000000f10055f0000000303000f01330001010100322063616665206c6174746520202020202020003200000005000000040101640100000100644b00010000000e0100000000320000080000000f32055f0000000000000f013d0001010000322063616665206372656d65202020202020200032000000050000000f0301640100000001641e00000000000a005a000000000000080000000f1000010000000000020a011400010000006d696c6b206672656520666c6f7720202020200032000000050000001b010164011400010964370001000000200050000000320000080014000f32055f3e00050300030f012f000102000063686f63696174746f20202020202020202020003200500005000000120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000130201640100000000642100010000000a0078003c00000000000000000f0f00010000000000030a01140001000000486f7420776174657220202020202020202020003200000005000000000a11000000000000001e00000000000a0000000000000000000000000f0f000000000000000000001400000000007832202020202020202020202020202020202000000000000a000000000a11000000000000001e00000000000a0000000000000000000000000f0f000000000000000000001400000000007832202020202020202020202020202020202000000000000a000000db"
tx = "f0f2050300010000000100814500"
def convert_hexstring_bytearray(hexstring):
    chunks = [tx[i:i+2] for i in range(0, len(tx), 2)]
    #print(chunks)
    dec = []
    for byte in chunks:
        dec.append(int(byte, 16))

    return dec

ar = convert_hexstring_bytearray(tx)
print(ar)



In [ ]:
import struct

# byte to value conversions

def convert_b_signed(byte):
    return struct.unpack('b', bytes.fromhex(byte))[0]

def convert_b_unsigned(byte):
    return struct.unpack('B', bytes.fromhex(byte))[0]

def convert_b_char(byte):
    return struct.unpack('c', bytes.fromhex(byte))[0]

# LITTLE ENDIAN
def convert_unsigned(hex):
    dec = int.from_bytes(bytes.fromhex(hex), byteorder="little")
    return dec

def convert_signed(hex):
    bits = len(hex) * 4
    dec  = convert_unsigned(hex)
    if dec > (bits-1)**2:
        dec = dec - bits**2
    return dec

def convert_2b_float(byte_2):
    return struct.unpack('e', bytes.fromhex(byte_2))[0]

def convert_4b_float(byte_4):
    return struct.unpack('f', bytes.fromhex(byte_4))[0]

def convert_8b_double(byte_8):
    return struct.unpack('d', bytes.fromhex(byte_8))[0]

def convert_string(byte):
    return struct.unpack('s', bytes.fromhex(byte))[0]



def convert_byte(byte):
    char = convert_b_char(byte)
    ub = convert_unsigned(byte)
    sb = convert_signed(byte)

    return [sb , ub, char]

def convert_2b(byte_2):
    u2 = convert_unsigned(byte_2)
    s2 = convert_signed(byte_2)
    float_s = convert_2b_float(byte_2)
    return [s2, u2, float_s ]

def convert_4b(byte_4):
    u4 = convert_unsigned(byte_4)
    s4 = convert_signed(byte_4)
    f = convert_4b_float(byte_4)

    return [ s4, u4, f]

def convert_8b(byte_8):
    u8 = convert_unsigned(byte_8)
    s8 = convert_signed(byte_8)
    double = convert_8b_double(byte_8)

    return [s8, u8, double]

# Windows hexstring into byte, b2, b4, b8 and converts data into typical data types
# Returns dataframes with the converted data (index indicated position of data in hexstring)
def convert_hexstring(data : str):
    # Window string into chunks of different byte sizes
    # Make window jump by 2 chars as 2chars = 1 byte
    b =  [data[(i*2):(i*2)+2] for i in range(0, int(len(data)/2))]
    b2 = [data[(i*2):(i*2)+4] for i in range(0, int(len(data)/2-1))]
    b4 = [data[(i*2):(i*2)+8] for i in range(0, int(len(data)/2-3))]
    b8 =  [data[(i*2):(i*2)+16] for i in range(0, int(len(data)/2-7))]


    b_conversion  = pd.DataFrame(columns=['byte',  'signed', 'unsigned', 'char'])
    b2_conversion = pd.DataFrame(columns=['2byte', 'signed', 'unsigned', 'float'])
    b4_conversion = pd.DataFrame(columns=['4byte', 'signed', 'unsigned', 'float'])
    b8_conversion = pd.DataFrame(columns=['8byte', 'signed', 'unsigned', 'double'])
 
    """
    conv = []
    for x in b_conversion['char'].values:
        try:
            a = str(x, 'utf-8')
            conv.append(a)
        except:
            conv.append("@")
    """
    string = ""
    
    b_conversion = b_conversion.drop('char', axis=1)

    for i, byte in enumerate(b):
        b_conversion.loc[i] = [byte] + convert_byte(byte)[0:2]

    for i, byte_2 in enumerate(b2):
        b2_conversion.loc[i] = [byte_2] + convert_2b(byte_2)
    
    for i, byte_4 in enumerate(b4):
        b4_conversion.loc[i] = [byte_4] + convert_4b(byte_4)
    
    for i, byte_8 in enumerate(b8):
        b8_conversion.loc[i] = [byte_8] + convert_8b(byte_8)

    return b_conversion, b2_conversion, b4_conversion, b8_conversion, string


cap = "f0f20503000100000001008145000003000000130009000000090111640114000103643700010000005500c8001e00320000080000000f10055f0000000303000f016300010100004c617474652020202020202020202020202020003200000005001501010111640114000104643800010000004600be002300320000080014000f10055f0000050303000f0134000102000043617070756363696e6f20202020202020202000320000000500d70009011164011400010364370001000000e60069000000320100080000000f32055f0000000303000f01410001000000576869746520436f666665652020202020202000320000000500330003011164010000010064390001000000180100000000320100080000000f32055f0000000000000f013e0001000000416d65726963616e6f2020202020202020202000320000000500c0001b011164011400010964340001000000230064000000320000080014000f10055f3400050300010f014100010200004d6f6368612020202020202020202020202020003200aa0005002901200511640100000009641e00000001000a0069000000000100080000000f1000012d00000000010a010a0001000000486f742043686f636f6c617465202020202020003200be0000000701120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000130211640100000000642100000000000a00b4000000000000080000000f1000010000000000020a01140001000000686f7420776174657220202020202020202020003200000005002f10120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000020111640114000104644c0001000000420096001e00320000080014000f32055f0000050303000f012b0001020000322043617070756363696e6f20202020202020003200000005000000080111640114000100645a0001000000460000000000320000080014000f32055f0000050000000f012300010200003220657370726573736f2020202020202020200032000000050000000a0111650114000103643d00010000003c00ae002d00320000080000000f10055f0000000303000f01330001010100322063616665206c6174746520202020202020003200000005000000040111640100000100644b00010000000e0100000000320000080000000f32055f0000000000000f013d0001010000322063616665206372656d65202020202020200032000000050000000f0311640100000001641e00000000000a005a000000000000080000000f1000010000000000020a011400010000006d696c6b206672656520666c6f7720202020200032000000050000001b011164011400010964370001000000200050000000320000080014000f32055f3e00050300030f012f000102000063686f63696174746f20202020202020202020003200500005000000120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000130211640100000000642100010000000a0078003c00000000000000000f0f00010000000000030a01140001000000486f7420776174657220202020202020202020003200000005000000000a11000000000000001e00000000000a0000000000000000000000000f0f000000000000000000001400000000007832202020202020202020202020202020202000000000000a000000000a11000000000000001e00000000000a0000000000000000000000000f0f000000000000000000001400000000007832202020202020202020202020202020202000000000000a00000015"
ame = "f016000500010000000100054902000000000000000000000057"

b, b2, b4, b8, s = convert_hexstring(ame)
print(s)



In [ ]:
def load_packets(filename):
    return pd.read_csv(f"./SERIAL_CAPTURE/{filename}", sep=',')

def load_samples(filename, sep):
    return pd.read_csv(f"./ENIGMA_SAMPLES/{filename}", sep=sep)

In [ ]:
#Repeats after ~22 packets therfore 250 packets contians 20 rows of data

packets = load_packets("errors_200.csv")#.iloc[0:500]

#Remove start bit and packet length data
packets['packet']  = packets['packet'].str.slice(6, 1000000)
#Seperate address out of packet
packets['address'] = packets['packet'].str.slice(0, 22)
#Remove address from packet
packets['packet']  = packets['packet'].str.slice(22, 1000000)


#Se merges to TX and RX packets into one row
combined  = packets[packets["dir"] == 'tx']
rx_packets = packets[packets["dir"] == 'rx']['packet']

combined = combined.rename(columns={'packet':'tx'})
combined = combined.drop(columns=['dir','len'])
combined['rx'] = rx_packets.values
packets = combined

starts_with = packets.apply(lambda row: row['rx'].startswith(row['rx'][0:]), axis=1)

solved_packets = packets


#packets.to_csv("mach_conf.csv", index=True)
nr          = [False for i in range(packets.shape[0])] 
f_nr        = [False for i in range(packets.shape[0])] 
info_txt_n  = [False for i in range(packets.shape[0])] 


In [ ]:
# REMOVED HEADER ADDED A COMMER FOR EXTRA ROW
sample_data = load_samples("errors_200.csv", ",")
#sample_data = sample_data[0:20]

"""
sample_data['comb_datum'] = sample_data['Datum'].astype(str) + ".2023::" + sample_data['Zeit'].astype(str)

sample_data['unixtime'] = pd.to_datetime(sample_data['comb_datum'], format='%d.%m.%Y::%H:%M:%S')
sample_data['unixtime_s'] =  sample_data['unixtime'].astype('uint64')   / 1000000000
sample_data['unixtime_ms'] = sample_data['unixtime'].astype('uint64')   / 1000000
sample_data['unixtime_ns'] = sample_data['unixtime'].astype('uint64')   / 1000
sample_data['unixtime_D'] =  sample_data['unixtime_s'].astype('uint64') / 86400
"""

In [ ]:
#Adresses which repeat in the data
addresses = ['0500010000000100004d01', '0500010000000100004d04', '0300010000000100004d00', '0300010000000100014d00', '0300010000000100014d05', '0300010000000100014d04', '0300010000000100014d03', '0300010000000100014d06', '0300010000000100014d07', '0300010000000100014d08', '0300010000000100014d09']


# Brainwave, compare packets with the same address
address_packets = packets.groupby('address')['rx'].apply(list).reset_index(name='packets')

address_packets['len'] = address_packets['packets'].apply(lambda x: len(x))
address_packets = address_packets[address_packets['len'] > 1]

def extraction(packet_list):
    usigned_byte = pd.DataFrame()
    for i, packet in enumerate(packet_list[0]):
        b, b2, b4, b8, s = convert_hexstring(packet)

        usigned_byte[f'Packet {i} S8']  = b['signed']
        usigned_byte[f'Packet {i} U8']  = b['unsigned']
        usigned_byte[f'Packet {i} S16'] = b2['signed']
        usigned_byte[f'Packet {i} U16'] = b2['unsigned']
        usigned_byte[f'Packet {i} S32'] = b4['signed']
        usigned_byte[f'Packet {i} U32'] = b4['unsigned']
        usigned_byte[f'Packet {i} S64'] = b8['signed']
        usigned_byte[f'Packet {i} U64'] = b8['unsigned']
    
    return usigned_byte

def tokenise_df(df):
    #This is so ugly
    tok_data = {}
    for column in df:
        tokens = {}
        token_data =[]
        token_id = 0
        for value in df[column]:
            if value not in tokens:
                token_data.append(token_id)
                tokens[value] = token_id
                token_id += 1
            else:
                token_data.append(tokens[value])
        
        tok_data[column] = token_data
    return pd.DataFrame(tok_data)

tok_sample = tokenise_df(sample_data)

address_df = {}
for address in addresses:
    mask = address_packets[address_packets['address'] == address]
    ex = extraction(mask['packets'].values)
    address_df[address] = {
        "S8" : ex.filter(regex='S8'),
        "U8" : ex.filter(regex='U8'),
        "T8"  : tokenise_df(ex.filter(regex='U8').T),
        "S16" : ex.filter(regex='S16'),
        "U16" : ex.filter(regex='U16'),
        "T16" : tokenise_df(ex.filter(regex='U16').T),
        "S32" : ex.filter(regex='S32'),
        "U32" : ex.filter(regex='U32'),
        "T32" : tokenise_df(ex.filter(regex='U32').T),
        "S64" : ex.filter(regex='S64'),
        "U64" : ex.filter(regex='U64'),
        "T64" : tokenise_df(ex.filter(regex='U64').T)
    }
    


In [42]:
#Works well for patterned data
token_df = tok_sample.add_prefix('sample_').T

for address in addresses:
    token_df = pd.concat([token_df, address_df[address]['T8'].add_prefix(f'{address}_T8_').T], axis=0)
    token_df = pd.concat([token_df, address_df[address]['T16'].add_prefix(f'{address}_T16_').T], axis=0)
    token_df = pd.concat([token_df, address_df[address]['T32'].add_prefix(f'{address}_T32_').T], axis=0)
    token_df = pd.concat([token_df, address_df[address]['T64'].add_prefix(f'{address}_T64_').T], axis=0)

a=[]
for index, row in token_df.iterrows():
    if all(np.array(row.values) == np.array(token_df.loc['sample_Info-Txt'].values)):
        a.append(f"Found Info-Txt at index {index}")

        
packets[packets['address'] == '0300010000000100014d00'].to_csv('./ENIGMA_OUTPUT/error_0300010000000100014d00')
#itterate over token_df and compare with sample_data
#token_df = token_df.T
#token_df = token_df.reset_index(drop=True)
#token_df = token_df.T

```text
TX Packet Makup - +26bytes

b0  -> (1 byte) f0 start bit
b1
b2  -> (2 bytes) Length of packet 
b3
b19 -> (16 bytes) Address / Command 
b20
b25 -> (5 Bytes .. imagine no upper limit) Fn Data


RX Packet Makup

b0  -> (1 byte) f0 start bit
b1
b2  -> (2 bytes) Length of packet 
b3
b19 -> (16 bytes) Address / Command  - Always returns same as TX
b20
bXX -> (XX Bytes .. can be thousands) Response data .. Some of this is always the same as TX?
```

Read Errors 

```Text
TX order of packets
Preamble? 

0300010000000100865200 - 0001000000000000000000de     *** RX contains num errors?
Data
Byte                                              25  
Desc                                              No 
                                                  $$$$
000100000000000000000014003a003c00ffffffffffff7701c80005007000000400003300000000004e

0500010000000100004d02 - 0000000000000000000000055b
0500010000000100004d03 - 00000000000000000000000b62

Error number 1
0500010000000100004d01 - 00000000000000000000000156   ***
0500010000000100004d04 - 0000000000000000000000010059 ???
0300010000000100004d00 - 000100000000000000000053     *** 
0300010000000100014d00 - 000100010000000b00000060     ??? RX contains all data
0300010000000100014d05 - 000000000000000000000058     ***
0300010000000100014d04 - 000000000000000000000057     ***
0300010000000100014d03 - 000000000000000000000056     ***
0300010000000100014d06 - 000000000000000000000059     ***
0300010000000100014d07 - 00000000000000000000005a     ***
0300010000000100014d08 - 00000000000000000000005b     ***
0300010000000100014d09 - 00000000000000000000005c     ***


Error number 2
0500010000000100004d01 - 00000000000000000000000156
0500010000000100004d04 - 000000000000000000000002005a
0300010000000100004d00 - 000100000000000000000053
0300010000000100014d00 - 000100020000000b00000061
0300010000000100014d05 - 000000000000000000000058
0300010000000100014d04 - 000000000000000000000057
0300010000000100014d03 - 000000000000000000000056
0300010000000100014d06 - 000000000000000000000059
0300010000000100014d07 - 00000000000000000000005a
0300010000000100014d08 - 00000000000000000000005b
0300010000000100014d09 - 00000000000000000000005c

Error number 3
0500010000000100004d01 - 00000000000000000000000156
0500010000000100004d04 - 000000000000000000000003005b
0300010000000100004d00 - 000100000000000000000053
0300010000000100014d00 - 000100030000000b00000062
0300010000000100014d05 - 000000000000000000000058
0300010000000100014d04 - 000000000000000000000057
0300010000000100014d03 - 000000000000000000000056
0300010000000100014d06 - 000000000000000000000059
0300010000000100014d07 - 00000000000000000000005a
0300010000000100014d08 - 00000000000000000000005b
0300010000000100014d09 - 00000000000000000000005c


...

No postamble

```

```Text
TX 
Address
0300010000000100014d00
Fn Data etc...
000100010000000b00000060
000100020000000b00000061
000100030000000b00000062

RX
Address
0300010000000100014d00
Data - all 2 byte little endian

Byte  3               11  13  15  17  19  21  23  25  27
Desc  Nr              Fnr FlagSec Min Hr  Day Mon Anz Inf Text-Info Buffer
      @@@@            !!!!$$$$@@@@!!!!$$$$@@@@!!!!$$$$@@@@.........
000100020000000b0000003e0001001e0031000d001900020004001a00000000000000000000000000000000000000000000000000000000000000000035
```

Fehlertext is mapped from F-Nr

Info-Txt ?? not useful data anyway


In [62]:
preamble = packets.iloc[0:3]
'0300010000000100865200'

In [ ]:
def read_packets(packets, address, index_of_data, bytes, conversion):
    #Mask out packets with wrong packet address
    p = packets[packets['address'] == address]
    data = []
    for index, row in p.iterrows():  
        data.append(conversion(row['rx'][index_of_data:index_of_data+bytes*2]))
    
    return data

#Read f-nr
data = read_packets(packets, "0300010000000100014d08", 22, 1, convert_b_unsigned)



In [61]:
# Think datetime is contained within 

dt = '00080010000d0019000200'

# Window in 32 bit & 64 bit numbers
b4 = [dt[(i*2):(i*2)+8] for i in range(0, int(len(dt)/2-3))]
b8 =  [dt[(i*2):(i*2)+16] for i in range(0, int(len(dt)/2-7))]

for b in b4: 
    dec = int.from_bytes(bytes.fromhex(b), byteorder="little")
    print(dec)
    try:
        date = datetime.datetime.fromtimestamp(dec)
        print(date)
    except:
        print(f"Failed to convert {b} to datetime")


268437504
1978-07-04 22:58:24
1048584
1970-01-13 04:16:24
218107904
1976-11-29 09:31:44
851984
1970-01-10 21:39:44
419433728
1983-04-17 14:22:08
1638413
1970-01-20 00:06:53
33560832
1971-01-24 11:27:12
131097
1970-01-02 13:24:57


In [ ]:
def compare_data(sample_series, read_series):
    if len(sample_series) != len(read_series):
        print("Length of series does not match")
        return 

    return pd.DataFrame({'sample': sample_series, 'read': read_series, "compare": sample_series == read_series})

print(sample_data['Anzahl'].values)

ret = compare_data(sample_data['Anzahl'].values, data)


